## Import

In [ ]:
import importlib
from typing import TypeVar

import torch
from torch_geometric import data, loader
import numpy as np

from awe import utils, filtering, features, html_utils, awe_graph, visual
from awe.data import swde, live, dataset

for module in [utils, filtering, dataset, swde, live, features, html_utils, awe_graph, visual]:
    importlib.reload(module)

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
sds = swde.Dataset(suffix='-exact')
#sds.validate()
ds = dataset.DatasetCollection()

## Split data

In [ ]:
T = TypeVar('T')
def train_val_split(data: list[T], val_split: float):
    split = int(np.floor(val_split * len(data)))
    copy = list(data)
    np.random.seed(42)
    np.random.shuffle(copy)
    return copy[split:], copy[:split]

In [ ]:
vertical = sds.verticals[0]
#train_pages, val_pages = train_val_split(website.pages[:100], .2)
train_pages = vertical.websites[0].pages[:300]
val_pages = vertical.websites[0].pages[300:400]
len(train_pages), len(val_pages)

## Prepare datapoints

In [ ]:
we = we if 'we' in locals() else features.WordEmbedding()
we.dimension

In [ ]:
ds.features = [
    features.Depth(),
    features.IsLeaf(),
    features.CharCategories(),
    we
]

In [ ]:
ds.feature_dim, ds.feature_labels

In [ ]:
ds.create('train', train_pages)
# Uncomment next line to invalidate computed features.
#ds['train'].delete_saved()

In [ ]:
ds['train'].prepare()

In [ ]:
ds.create('val', val_pages)
# Uncomment next line to invalidate computed features.
#ds['val'].delete_saved()

In [ ]:
ds['val'].prepare()

In [ ]:
ds.first_dataset.label_map

## Create dataloaders

In [ ]:
BATCH_SIZE = 1
ds['train'].loader = loader.DataLoader(ds['train'], batch_size=BATCH_SIZE, shuffle=True)
ds['val'].loader = loader.DataLoader(ds['val'], batch_size=BATCH_SIZE)

In [ ]:
for batch in ds['train'].loader:
    print(batch)
    break

In [ ]:
len(ds['train'].loader) + len(ds['val'].loader)

## Inspect data

In [ ]:
interesting_nodes = (
    (node, x, y)
    for node, x, y in ds['val'].iterate_data()
    if node.labels == ['price']
)
next(iter(interesting_nodes))

## Weight labels

In [ ]:
def count_label(data: list[data.Data], label: int):
    return len([1 for d in data for y in d.y if y == label])

def count_labels(data: list[data.Data]):
    return [count_label(data, label) for label in ds.first_dataset.label_map.values()]

In [ ]:
#label_counts = count_labels(ds.data['train'])
#label_counts, len(ds.data['train'])

In [ ]:
#label_weights = [len(ds.data['train']) / count for count in label_counts]
#label_weights

In [ ]:
# Manual override
label_count = len(ds.first_dataset.label_map)
label_weights = [1] + [100_000] * (label_count - 1)
label_weights

## Train a model

In [ ]:
import pytorch_lightning as pl

from awe import awe_model, gym, utils
for module in [awe_model, gym, utils]:
    importlib.reload(module)

In [ ]:
model = awe_model.AweModel(ds.feature_dim, label_count, label_weights)
model

In [ ]:
g = gym.Gym(ds, model)
# Uncomment next line to re-train a model.
#g.restore_checkpoint = False
g.get_last_checkpoint_path(), g.get_last_checkpoint_version()

In [ ]:
g.trainer = pl.Trainer(
    max_epochs=50,
    progress_bar_refresh_rate=100,
    resume_from_checkpoint=g.get_last_checkpoint_path(),
    logger=g.create_logger(),
)

In [ ]:
g.trainer.logger.version

In [ ]:
g.trainer.fit(model, ds['train'].loader, ds['val'].loader)

In [ ]:
# TODO: Breaks torch_geometric for some reason!
#g.save_model()

In [ ]:
g.save_model_text()

In [ ]:
g.save_results('val')

In [ ]:
g.save_pages()

In [ ]:
# Uncomment and change name to save interesting results.
#g.save_named_version('one-website')

## Example prediction

In [ ]:
from awe import predictor
importlib.reload(predictor)

In [ ]:
val_predictor = predictor.Predictor(ds, 'val', model)

In [ ]:
val_predictor.evaluate(range(5))

In [ ]:
val_predictor.get_example_texts(range(5))

## Live prediction

In [ ]:
urls = [
    'https://www.cars.com/vehicledetail/81d8ee1f-155e-44ec-8ea4-0b25b0ca608a/'
]
live_pages = [live.Page(url) for url in urls]

In [ ]:
# Download pages.
[page.dom for page in live_pages]

In [ ]:
ds.create('live', live_pages).prepare(skip_existing=False)

In [ ]:
live_predictor = predictor.Predictor(ds, 'live', model)

In [ ]:
live_predictor.get_example_texts(range(len(live_pages)))